# LN - Data PP - Stability and efficiency calculations

## Import libraries and parameters

In [34]:
import sqlite3
import numpy as np
import pandas as pd
import networkx as nx
import itertools
#import matplotlib.pyplot as plt
import time
import pickle

import os
import re
import sys
import io
import random
from itertools import islice
import math

from tqdm.notebook import trange, tqdm
from time import sleep

from dask_cloudprovider import FargateCluster
from dask.distributed import Client
import dask.array as da
import dask
dask.config.set({'distributed.scheduler.allowed-failures': 50}) 


import boto3


In [2]:
# Define parameters

bucket='ln-strategy-data'
extraction_id=1587447789
#extraction_id=1585344554

## Connection to AWS Resources

### S3

In [3]:
# Initiate s3 resource

session = boto3.session.Session()
s3 = session.resource('s3')

### Fargate Clusters

In [ ]:
#cluster = FargateCluster(n_workers=100,scheduler_timeout='20 minutes',image='dsrincon/dask-graph:nx-scipy-v1',scheduler_cpu=4096,scheduler_mem=16384,worker_mem=32768)
cluster = FargateCluster(n_workers=100,scheduler_timeout='20 minutes',image='dsrincon/dask-graph:nx-scipy-v1',scheduler_cpu=4096,scheduler_mem=16384)

In [16]:
cluster

In [58]:
#client = Client(cluster)
cluster=Client('tcp://3.214.224.107:8786')

## Define Utility functions

### Save Data

**FUNCTION: Write output to DataFrame**

In [7]:
# Function write output to DataFrame

"""
Function
--------
add_columns
    Function that takes an output from a decision comparisson computation and adds it's results for nodes 1 and 0 in the main DataFrame

Parameters
----------
output_diclist : list
    Dictionary of the form (node0_dic_i,node1_dic_i) where i runs for all of the blocks being compared. 

original_df: Pandas DataFrame
    Original DataFrame containing the opening and closure information for each channel, with a column named 'short_channel_id' to denote 
    id of channel. 

column_name_node0: string
    Name for column in dataframe where the results will be stored for node 0
    
column_name_node1: string
    Name for column in dataframe where the results will be stored for node 1

    
Returns
-------
no_changes: list
    List with the 'short_channel_id' of the channels edited. 
 
"""


def add_columns(output_diclist,original_df,column_name_node0,column_name_node1):


    # Merge individual dictionaries into one for each node
    node0_dic={}
    node1_dic={}
    for dic_tuple in output_diclist:
        node0_dic.update(dic_tuple[0])
        node1_dic.update(dic_tuple[1])
    
    # Add to DataFrame

    # Create empty columns
    original_df[column_name_node0]=np.nan
    original_df[column_name_node1]=np.nan

    # Populate df with values
    original_df[column_name_node0]=original_df['short_channel_id'].map(node0_dic)
    original_df[column_name_node1]=original_df['short_channel_id'].map(node1_dic)
    
    # Calculate values changed
    rows_edited=(original_df[original_df[column_name_node0].notnull()]['short_channel_id']).tolist()
    
    return rows_edited


**FUNCTION: Save python object to S3 using pickle**

In [8]:
# Function to write pickle data to S3 bucket

"""
Function
--------
pickle_save_s3
    Saves Python object to S3 as pickle object

Parameters
----------
obj : <any>
    Python Object

blocks: list
    List of extracted blocks

extraction_id: int
    Number of block extraction
    
name: string
    Name of object to add to filename in S3

    
Returns
-------
response: int
    HTTP response code from S3 
 
"""


def pickle_save_s3(obj,blocks,extraction_id,name):


    # Define number of blocks
    start_block=np.min(np.array(blocks))
    end_block=np.max(np.array(blocks))
    no_blocks=len(blocks)

    # Load S3 and bucket details
    session = boto3.session.Session()
    s3 = session.resource('s3')

    # File path and name ([extraction_id][name]-[no_blocks]-[start_block]-[end_block])
    key='graph_snapshots/'+str(extraction_id)+'_connected/.data_transformations/'+str(extraction_id)+name+'-'+str(no_blocks)+'-'+str(start_block)+'-'+str(end_block)+'.pkl'

    # Create pickle object and send to S3
    pickle_byte_obj = pickle.dumps(obj) 
    response=s3.Object(bucket,key).put(Body=pickle_byte_obj)
    
    return response['ResponseMetadata']

### Load Data

**FUNCTION: Load single graph**

In [9]:


"""
Function
--------
load_graph
    Loads networkX (pickle serialized) object from S3

Parameters
----------
key : str
    Path in S3 bucket for individual pickled serialized networkX graph object 

    
Returns
-------
response: networkX graph
    Graph object
 
"""

def load_graph(key):
    session = boto3.session.Session()
    s3 = session.resource('s3')
    response = s3.Object(bucket_name=bucket, key=key).get()
    G=pickle.loads(response['Body'].read())
    
    return G
   

## Load Data

In [10]:
# Load objects form S3
# Dataframe

decisions_load = s3.Object(bucket_name=bucket, key='decisions_df.csv').get()
decisions_df=pd.read_csv(io.BytesIO(decisions_load['Body'].read()))

# Channel closures
closure_file = s3.Object(bucket_name=bucket, key='channel_closures.p').get()
channel_closures = pickle.loads(closure_file['Body'].read())
    
    
# Channel openings 
opens_file = s3.Object(bucket_name=bucket, key='channel_opens.p').get()
channel_opens = pickle.loads(opens_file['Body'].read())

    

# Create list with graph keys

#TODO: Save graphs as numpy array in single H5 file to reduce. Test if creating graphs takes longer than reading from S3

# graph_dir='./data/graph_snapshots' - For local tests


graph_keys = [obj.key 
        for obj in s3.Bucket(name=bucket).objects.all()
        if re.match(".*"+str(extraction_id)+"_connected/.*\.gpickle",obj.key)]




In [17]:
# Define Blocks to be extracted and define graph


# Base lists to be populated
graph_snapshots=[]
blocks=[]


extract_keys=graph_keys[6:] # Blocks below 6th index are <3 and affect some graph metrics

for key in extract_keys: # Change to [700:] for full range
    
    # Create block list from file_names
    block_i=int(key.split(".")[0].split("/")[-1]) 
    blocks.append(block_i)
    
    #Extract graphs - UNCOMMENT TO have them out of function
    #G=dask.delayed(load_graph)(key)
    #graph_snapshots.append(G)
    
   
start_block=np.min(np.array(blocks))
end_block=np.max(np.array(blocks))
no_blocks=len(blocks)

    

In [12]:
# Test: extracted formats
print("Number of total graph keys:{}".format(len(graph_keys)))
print("Number of blocks to be processed:{}".format(len(extract_keys)))
print("---Sample graph keys---")
print(graph_keys[0])
print("---Sample channel opens---")
print(channel_opens[513675])
print("---Sample channel closures---")
print(channel_closures[592638])



Number of total graph keys:36542
Number of blocks to be processed:994
---Sample graph keys---
graph_snapshots/1587447789_connected/505149.gpickle
---Sample channel opens---
[(7521, 6156, {'capacity': 50000, 'open_fee': 306, 'dec_id': 1, 'channel_id': '513675x2245x0', 'no_channels': 0})]
---Sample channel closures---
[(2643, 6038, {'close_type': 'force', 'dec_id': 26620, 'channel_id': '570913x720x1', 'capacity': 300000}), (6038, 5314, {'close_type': 'mutual', 'dec_id': 0, 'channel_id': '505149x622x0', 'capacity': 300000})]


In [4]:
# Sort and visualize DataFrame

decisions_df.sort_values(by=['open_block'],inplace=True,ascending=True)
print('Size of DataFrame in Memory:{}'.format(sys.getsizeof(decisions_df)))
# Check specific channel id
#decisions_df[decisions_df['short_channel_id']=='513675x2245x0'].head()

decisions_df.head()

Size of DataFrame in Memory:64821774


,Unnamed: 0,short_channel_id,open_block,open_transaction,address,close_block,close_transaction,node0,node1,satoshis,...,close_fee,last_update,close_type,close_htlc_count,close_balance_a,close_balance_b,dec_id,node0_id,node1_id,node_pair
72475,0,505149x622x0,505149,f6bc767df9148ebf76d2b9baf4eb46e3230712c2bf5a51...,bc1qjmg6ev344fenh3zhg0yjl6hyvxpxluw6x9nn2a5lv4...,592638.0,82cb2ea2a06c8c453d8b9ca08e17bbefe87225aa380b2d...,0250373555232cec757ea141273e75381c84cc3ab22f1e...,02ef61a252f9504a42fc264a28476f44cea0711a44b2da...,300000,...,184.0,1.563172e+09,mutual,0.0,3570.0,296246.0,0,6038,5314,32085932
72474,38787,506402x1391x1,506402,2cdfc4fec2049d66a04fa5bdf468efb19c0354c60b8cf2...,bc1qvjx5t8y7j83udzuj38ukmqecv5d9jn762mchxkgvaf...,NaN,NaN,0313f9449cdb528dc9707c02da507cc9306eedc415091c...,035f1498c929d4cefba4701ae36a554691f526ff60b176...,1111934,...,NaN,NaN,NaN,NaN,NaN,NaN,38787,934,3023,2823482
72473,38788,506847x1633x0,506847,19ee11ce977facd380b92126834a3aca318f3cb905d99b...,bc1q29g43xrz9gujgt60gykzq3vh0ewfav7vmfqcnmf50u...,NaN,NaN,023d280ae29f84dcfd289eb66b57227fea3a7bde97ec28...,0273081ce642554d5a68a5236564fe88a3783457dc09e5...,40000,...,NaN,NaN,NaN,NaN,NaN,NaN,38788,3452,576,1988352
72472,38789,508075x1694x1,508075,e267e54872053a7618567f31a9d27e38cdbff0e4176144...,bc1qpxzqp2xyy0gzn8xwu6lqg3a66tuhsg5w849t0j5rdr...,NaN,NaN,03557fd11b58cb93d2ad4fab4dd4cff7462a97e21e8f6b...,03cbf298b068300be33f06c947b9d3f00a0f0e8089da32...,100000,...,NaN,NaN,NaN,NaN,NaN,NaN,38789,3436,3310,11373160
72471,58766,508090x1515x1,508090,33d645657de8a587137b8039e52452557d4279a3f47366...,bc1qneudwey0dpgy9nj2g8ech0lqqrhz52agcj984rs6zh...,616838.0,123777e4dfadc7c008a54c2d55b670067a58cdcbc8b2ec...,028314f021602092779aedd4ef39f3b5809f9b6046f8bc...,02d4531a2f2e6e5a9033d37d548cff4834a3898e74c3ab...,400000,...,4887.0,1.581152e+09,NaN,NaN,NaN,NaN,58766,2378,4223,10042294


**TESTS**

In [ ]:
# Test Lazy Graph extract
blocks_att=[]
for i in range(len(graph_snapshots)):
    graph_i=dask.compute(graph_snapshots[i])
    block=graph_i.graph['block']
    blocks_att.append(block)

print(blocks_att)

#graph_snapshots=dask.compute(*graph_snapshots)
#block=graph_snapshots[0].graph['block']
    
#print(len(graph_snapshots[5]))
#print(graph_snapshots[3].graph['block'])

# Delayed testing
#results = dask.compute(*futures)
#graphs=dask.compute(*graph_snapshots)


# Comparative Analysis

In order to understand the potential motivations behind each decision we analyze each decission (opening or closure of a channel) independently from the perspective of each of the participants in the decission, which we'll call the node under analysis. For each decission we extract or compute the following information: 

Betweenness centrality measures how central is a network to the flow of information in a network. In the case of the Lightning Network the higher the betweenness centrality of a node, the more transactions (messages) that are routed through it. In particular, we will use a measure of betweenness centrality defined in (Brandes and Fleischer 2005 - https://link.springer.com/chapter/10.1007/978-3-540-31856-9_44) that models infomation through a network, as electric current, efficiently and not only considering shortest path. This allows us to account for the fact that not all transactions travel through shortes path given that there are fee and capacity considerations.  

## Baseline Measurments

**FUNCTION: Property measurement for a collection of graphs**

In [13]:
### NEW
# Function to calculate base measurement for every graph in snapshot

"""
Function
--------
collection_measure
    Iterates over graph snapshots and calculates measurement for every node in each of the graphs.

Parameters
----------
g_snapshots : list
    List of delayed nx graph elements contianing graph snapshots
    

measurement: string
    Name of the type of measurment that will applied to the graph. (See graph_measurement function for options)



Returns
-------
snapshot_mes_dic: dic
    Dictionary with blocks as keys and dic of measurements as values

"""
def collection_measure(bucket,graph_keys,measurement):
    
    snapshot_mes_list=[]
    
    
    
   
    for key in graph_keys:
      
        
        measurement_input=(key,measurement,'capacity',bucket)
        
        b_g_tuple=dask.delayed(graph_measurement)(measurement_input)
        snapshot_mes_list.append(b_g_tuple)
        

    futures = dask.persist(*snapshot_mes_list)
    
    start=time.time()
    snapshot_mes_list = dask.compute(*futures)
    #snapshot_mes_list = dask.compute(*snapshot_mes_list)
    snapshot_mes_dic={record[0]:record[1] for record in snapshot_mes_list}
    end=time.time()
    print('Compute in seconds: {}'.format(end-start))
    
    return snapshot_mes_dic

In [ ]:
# Function to calculate base measurement for every graph in snapshot

"""
Function
--------
collection_measure
    Iterates over graph snapshots and calculates measurement for every node in each of the graphs.

Parameters
----------
g_snapshots : list
    List of delayed nx graph elements contianing graph snapshots
    

measurement: string
    Name of the type of measurment that will applied to the graph. (See graph_measurement function for options)



Returns
-------
snapshot_mes_dic: dic
    Dictionary with blocks as keys and dic of measurements as values


def collection_measure(g_snapshots,measurement):
    
    snapshot_mes_list=[]
   
    for g in graph_snapshots:
      
        
        measurement_input=(g,measurement,'capacity')
        
        b_g_tuple=dask.delayed(graph_measurement)(measurement_input)
        snapshot_mes_list.append(b_g_tuple)
        

    #futures = dask.persist(*snapshot_mes_list)
    
    start=time.time()
    #snapshot_mes_list = dask.compute(*futures)
    snapshot_mes_list = dask.compute(*snapshot_mes_list)
    snapshot_mes_dic={record[0]:record[1] for record in snapshot_mes_list}
    end=time.time()
    print('Compute in seconds: {}'.format(end-start))
    
    return snapshot_mes_dic
"""

**FUNCTION: Property measurment for a single graph**

In [60]:
###NEW
"""
Function
--------
graph_measurement
    Performs specific graph measurement 

Parameters
----------
measurment_input: tuple
    g : nx graph
        NetworkX graph object over which measurment will be calculated for each node

    measurment: string
        Type of measurement to be performed on graph
        
    weight: string
        Edge attribute to be used as weight
    
Returns
-------
g_dir: dir
    Dictionary with measurment values for each node
"""

def graph_measurement(measurment_input):
    
    # Extract inputs
    key=measurment_input[0]
    measurement=measurment_input[1]
    weight=measurment_input[2]
    bucket=measurment_input[3]
    
    # Download graph
    session = boto3.session.Session()
    s3 = session.resource('s3')
    response = s3.Object(bucket_name=bucket, key=key).get()
    g=pickle.loads(response['Body'].read())
    
    
    if measurement=='current_betweeness_full':
        g_dir=nx.algorithms.centrality.current_flow_betweenness_centrality(g,weight=weight)
    
    elif measurement=='current_betweeness':
        g_dir=nx.algorithms.centrality.approximate_current_flow_betweenness_centrality(g,weight=weight)
    
    elif measurement=='current_closeness':
        g_dir=nx.algorithms.centrality.current_flow_closeness_centrality(g,weight=weight)
    
    elif measurement=='closeness':
        g_dir=nx.closeness_centrality(g)
        
    elif measurement=='clustering':
        g_dir=nx.clustering(g,weight=weight)
        
    elif measurement=='node_count':
        g_dir=len(g.nodes())
        
    elif measurement=='channels':
        g_dir=dict(list(g.degree(g.nodes())))
    
    elif measurement=='capacity':
        g_dir=dict(list(g.degree(g.nodes(),weight=weight)))
        
    elif measurement=='age': 
        pass
    elif measurement=='growth': 
        pass
    
    
    # Safe graph processing to S3
    
    extraction_id=key.split('/')[1].split('_')[0]
    block=key.split('/')[2].split('.')[0]
    key_out='graph_snapshots/'+extraction_id+'_connected/.data_transformations/'+measurement+'/raw_score/'+block+'.pkl'
    pickle_byte_obj = pickle.dumps(g_dir) 
    response=s3.Object(bucket,key_out).put(Body=pickle_byte_obj)['ResponseMetadata']['HTTPStatusCode']
    
    block=g.graph['block']
    
    return (block,response)

In [29]:
key='graph_snapshots/1587447789_connected/505149.gpickle'
extraction_id=key.split('/')[1].split('_')[0]
block=key.split('/')[2].split('.')[0]
measurement='current_betweeness_full'
key_out='graph_snapshots/'+extraction_id+'_connected/.data_transformations/'+measurement+'/'+block+'.pkl'
print (key_out)

graph_snapshots/1587447789_connected/.data_transformations/current_betweeness_full/505149.pkl


In [12]:

"""
Function
--------
graph_measurement
    Performs specific graph measurement 

Parameters
----------
measurment_input: tuple
    g : nx graph
        NetworkX graph object over which measurment will be calculated for each node

    measurment: string
        Type of measurement to be performed on graph
        
    weight: string
        Edge attribute to be used as weight
    
Returns
-------
g_dir: dir
    Dictionary with measurment values for each node


def graph_measurement(measurment_input):
    
    g=measurment_input[0]
    measurement=measurment_input[1]
    weight=measurment_input[2]
    
    if measurement=='current_betweeness':
        g_dir=nx.algorithms.centrality.current_flow_betweenness_centrality(g,weight=weight)
    
    elif measurement=='current_closeness':
        g_dir=nx.algorithms.centrality.current_flow_closeness_centrality(g,weight=weight)
    
    block=g.graph['block']
    
    return (block,g_dir)
    
"""

**FUNCTION: Property measurment for a couple of nodes in a graph**

In [15]:

"""
Function
--------
node_measurement
    Performs selected graph measurment on specific nodes in graph

Parameters
----------
g : nx graph
    NetworkX graph object over which the measurment will be performed

measurement: string
    Type of measurement to be performend in graph
    
node0: int
    Node id for node 0

node1: int
    Node id for node 1
    
Returns
-------
node_tuple: tuple
    Tuple of the form (node0_mes,node1_mes)
    
    node0_mes: float
        Graph measurement for node0
    node1_mes: float
        Graph measurement for node1
"""


        
    
    

def node_measurement(g,measurement,node0,node1):
    
    measurement_input=(g,measurement,'capacity')
    block,g_dir=graph_measurement(measurement_input)
    
    node0_mes=g_dir[node0]
    node1_mes=g_dir[node1]
        
    # Update marginal values for node0 and node1
        
    if (g.has_node(node0)): #If connected component of marginal graph contains node0 find betweeness
        node0_mes=g_dir[node0]
    else: # else update with fixed value
        node0_mes=0
            
    if (g.has_node(node1)): #If connected component of marginal graph contains node1 find betweeness
        node1_mes=g_dir[node1]
    else: # else update with fixed value
        node1_mes=0
    
    return (node0_mes,node1_mes)
    
    

In [ ]:
def incremental_closeness(bucket,graph_keys,blocks):
    
    session = boto3.session.Session()
    s3 = session.resource('s3')
    counter=0
    
    
    # Channel closures
    closure_file = s3.Object(bucket_name=bucket, key='channel_closures.p').get()
    channel_closures = pickle.loads(closure_file['Body'].read())
    
    
    # Channel openings 
    opens_file = s3.Object(bucket_name=bucket, key='channel_opens.p').get()
    channel_opens = pickle.loads(opens_file['Body'].read())

    
    
    
    for i  in range(1,len(graph_keys)):
        
        graph_keys
        
        response = s3.Object(bucket_name=bucket, key=key).get()
        G=pickle.loads(response['Body'].read())
        block=g.graph['block']
        new_edges=list(itertools.chain.from_iterable([opens for block_key,opens in channel_opens.items() if block_key==block]))
        closed_edges=list(itertools.chain.from_iterable([closures for block_key,closures in channel_closures.items() if block_key==block])) 
    
        
        if counter==0:
            closeness=nx.closeness_centrality(g)

        else:


            # Add edges from CHANNEL OPENS

            for edge in new_edges:

                #If edge existed, update value and channel counter
                if G.has_edge(edge[0],edge[1]):

                    base_capacity=G.edges[edge[0],edge[1]]['capacity']
                    added_capacity=edge[2]['capacity']
                    G.edges[edge[0],edge[1]]['capacity']=base_capacity+added_capacity

                else:
                    G.add_edges_from([edge])

                # Increase channel counter for edge
                G.edges[edge[0],edge[1]]['no_channels']+=1



            # Remove edges from CHANNEL CLOSURES

            ## Create temp list of closures that removes metadata to remove from graph and apply modifications
            #closure_pairs=[(closure[0],closure[1]) for closure in closed_edges]


            for edge in closed_edges:

                # Verify if existing edges result from multiple channels, if so, only reduce capacity otherwise remove edge

                no_channels=G.edges[edge[0],edge[1]]['no_channels']
                if no_channels>1:
                    base_capacity=G.edges[edge[0],edge[1]]['capacity']
                    reduced_capacity=edge[2]['capacity']
                    G.edges[edge[0],edge[1]]['capacity']=base_capacity-reduced_capacity
                    G.edges[edge[0],edge[1]]['no_channels']-=1 # Decrease counter
                else:
                    G.remove_edges_from([(edge[0],edge[1])])
    
    

In [23]:


measurement_input=(extract_keys[35000],'clustering','capacity',bucket)

start=time.time()
test_aprox = graph_measurement(measurement_input)
end=time.time()

print('With aproximate betweeness: {}s '.format(end-start))

'''
measurement_input_2=(extract_keys[10000],'current_betweeness_full','capacity',bucket)

start=time.time()
test_full = graph_measurement(measurement_input_2)
end=time.time()

print('With full betweeness: {}s '.format(end-start))
'''


With aproximate betweeness: 3.782655715942383s 


"\nmeasurement_input_2=(extract_keys[10000],'current_betweeness_full','capacity',bucket)\n\nstart=time.time()\ntest_full = graph_measurement(measurement_input_2)\nend=time.time()\n\nprint('With full betweeness: {}s '.format(end-start))\n"

In [32]:
snapshot_nodes=collection_measure(bucket,extract_keys,'node_count')

Compute in seconds: 10.438627481460571


In [ ]:
sorted(snapshot_nodes.items())[:100]

**SCRIPT: Baseline Current Betweeness**

In [40]:
# Calculate Baseline betweeness
snapshot_bet=collection_measure(bucket,extract_keys,'current_betweeness')


Compute in seconds: 8009.403836488724


In [41]:
# Save baseline current betweeness to S3
response=pickle_save_s3(snapshot_bet,blocks,extraction_id,'snapshot_bet')
if response['HTTPStatusCode']==200:
    print('Save to S3 succesful')
else:
    print(response)

Save to S3 succesful


**SCRIPT: Baseline Current Closeness**

In [ ]:
# Calculate Baseline current closeness
snapshot_clo=collection_measure(bucket,extract_keys,'current_closeness')

In [ ]:
snapshot_clo=collection_measure(bucket,extract_keys,'closeness')

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError


In [ ]:
# TEST all saved correctly
responses=np.array([r for b,r in snapshot_clo.items()])

if np.amax(responses)==np.amin(responses) and len(responses) == len(extract_keys):
    print('Save to S3 succesful')
else:
    print('Error saving to S3')
    
measurement='current_closeness'    
rand_block=str(random.choice(blocks))    
test_key='graph_snapshots/'+str(extraction_id)+'_connected/.data_transformations/'+measurement+'/raw_score/'+rand_block+'.pkl'
test_file = s3.Object(bucket_name=bucket, key=test_key).get()
test_object = pickle.loads(test_file['Body'].read())
print('Example of {} saved for block {}:'.format(measurement,rand_block))
print(test_object.items()[:10])    
print('Total entries: {}'.format(len(list(test_object.items()))))

**SCRIPT: Baseline Channels**

In [44]:
# Calculate Baseline degree
snapshot_channels=collection_measure(bucket,extract_keys,'channels')

Compute in seconds: 186.33308386802673


In [50]:
# TEST all saved correctly
responses=np.array([r for b,r in snapshot_channels.items()])

if np.amax(responses)==np.amin(responses) and len(responses) == len(extract_keys):
    print('Save to S3 succesful')
else:
    print('Error saving to S3')
    
measurement='channels'    
rand_block=str(random.choice(blocks))    
test_key='graph_snapshots/'+str(extraction_id)+'_connected/.data_transformations/'+measurement+'/raw_score/'+rand_block+'.pkl'
test_file = s3.Object(bucket_name=bucket, key=test_key).get()
test_object = pickle.loads(test_file['Body'].read())
print('Example of {} saved for block {}:'.format(measurement,rand_block))
print(list(test_object.items())[:10])    
print('Total entries: {}'.format(len(list(test_object.items()))))

Save to S3 succesful
Example of channels saved for block 593261:
[(5314, 7), (934, 3), (3023, 1), (3452, 4), (576, 1), (3436, 38), (3310, 1), (2378, 1), (4223, 9), (422, 39)]
Total entries: 5154


**SCRIPT: Baseline Capacity**

In [45]:
# Calculate Baseline degree
snapshot_capacity=collection_measure(bucket,extract_keys,'capacity')

Compute in seconds: 205.32206177711487


In [51]:
# TEST all saved correctly
responses=np.array([r for b,r in snapshot_capacity.items()])

if np.amax(responses)==np.amin(responses) and len(responses) == len(extract_keys):
    print('Save to S3 succesful')
else:
    print('Error saving to S3')
    
measurement='capacity'    
rand_block=str(random.choice(blocks))    
test_key='graph_snapshots/'+str(extraction_id)+'_connected/.data_transformations/'+measurement+'/raw_score/'+rand_block+'.pkl'
test_file = s3.Object(bucket_name=bucket, key=test_key).get()
test_object = pickle.loads(test_file['Body'].read())
print('Example of {} saved for block {}:'.format(measurement,rand_block))
print(list(test_object.items())[:10])
print('Total entries: {}'.format(len(list(test_object.items()))))

Save to S3 succesful
Example of capacity saved for block 586042:
[(6038, 8878679), (5314, 14026340), (934, 1171934), (3023, 1111934), (3452, 2063908), (576, 40000), (3436, 6948131), (3310, 100000), (2378, 400000), (4223, 24298841)]
Total entries: 4920


## Comparissons

**FUNCTION: Compare property changes for nodes in a collection of graphs**

In [ ]:
###NEW 

"""
Function
--------
collection_compare
    Iterates over blocks to calculate marginal change in metric for nodes that made decisions (opens/closures)

Parameters
----------
blocks : list
    List of blocks extracted when reading graphs
    
    
dec_dic: dic
    Dictionary with channel decisions (open or closure) per block
    
    
graph_snapshots: list
    List of delayed dask objects each pointing to a graph snapshot to be loaded from S3

snapshots_base: dic
    Dictionary of dictionaries containing the base measurments per block per node

measurement: string
    Name of the type of measurment that will applied to the graph
    
type_dec: string
    The type of decisions that will be analyzed 'opens' or 'closures'
    

Returns
-------
futures: list
    List of tuples of the form (dic_node0,dic_node1) where dic_node0/1 is a dictionary containing the marginal changes for node0/1 
    for every decision in a given block. The dictionaries are future dask objects that still need to be explicitly computed. 

"""
def collection_compare(blocks,dec_dic,graph_keys,snapshots_base,measurement,type_dec):

    futures_list=[] # list to populate with futures per block
    
    with tqdm(total=len(range(1,len(graph_keys)))) as pbar:
        for i in range(1,len(graph_keys)): # iterate through blocks

            # extract information from parameters and construct input tuple to delayed function
            block=blocks[i]
            block_prev=blocks[i-1]
            block_dec=dec_dic[block]
            key=graph_keys[i-1] # Pass previous graph
            block_base=snapshots_base[block_prev]
            block_res=snapshots_base[block]
            input_tuple=(block,g,block_dec,block_base,measurement,type_dec,block_res)
            
            # submit to delayed function and append to list
            output_tuple=dask.delayed(graph_compare)(input_tuple)
            futures_list.append(output_tuple)
            pbar.update(1)

    # persist to disk and return
    futures = dask.persist(*futures_list)
    return futures
    

In [ ]:

"""
Function
--------
collection_compare
    Iterates over blocks to calculate marginal change in metric for nodes that made decisions (opens/closures)

Parameters
----------
blocks : list
    List of blocks extracted when reading graphs
    
    
dec_dic: dic
    Dictionary with channel decisions (open or closure) per block
    
    
graph_snapshots: list
    List of delayed dask objects each pointing to a graph snapshot to be loaded from S3

snapshots_base: dic
    Dictionary of dictionaries containing the base measurments per block per node

measurement: string
    Name of the type of measurment that will applied to the graph
    
type_dec: string
    The type of decisions that will be analyzed 'opens' or 'closures'
    

Returns
-------
futures: list
    List of tuples of the form (dic_node0,dic_node1) where dic_node0/1 is a dictionary containing the marginal changes for node0/1 
    for every decision in a given block. The dictionaries are future dask objects that still need to be explicitly computed. 


def collection_compare(blocks,dec_dic,graph_snapshots,snapshots_base,measurement,type_dec):

    futures_list=[] # list to populate with futures per block
    
    with tqdm(total=len(range(1,len(graph_snapshots)))) as pbar:
        for i in range(1,len(graph_snapshots)): # iterate through blocks

            # extract information from parameters and construct input tuple to delayed function
            block=blocks[i]
            block_prev=blocks[i-1]
            block_dec=dec_dic[block]
            g=graph_snapshots[i-1] # Pass previous graph
            block_base=snapshots_base[block_prev]
            block_res=snapshots_base[block]
            input_tuple=(block,g,block_dec,block_base,measurement,type_dec,block_res)
            
            # submit to delayed function and append to list
            output_tuple=dask.delayed(graph_compare)(input_tuple)
            futures_list.append(output_tuple)
            pbar.update(1)

    # persist to disk and return
    futures = dask.persist(*futures_list)
    return futures
"""    

**FUNCTION: Compare property changes for nodes in a graph**

In [ ]:

"""
Function
--------
graph_compare
    Calculates marginal change in metric for node0, node1 make decisions (open/close channels) in a single block

Parameters
----------
input_tuple : tuple
    
    block: int
        Block number
    g: nx_graph 
        Graph snapshot (as dask delayed object)
    block_dec: list
        List of tuples in nx edge format (u,v,att_dic) for all the decisions (channel opens/closures) made in that block  
    block_base: dic
        Dictionary of base measurements for each node in the graph snapshot corresponding to that block
    measurement: string
        Name of measurement to be computed
    type_dec: string
    The type of decisions that will be analyzed 'opens' or 'closures'
    
    block_res: dic
        Dictionary of base measurements for each node in the graph snapshot corresponding to the next block
    

Returns
-------
nodes_mar_dic: tuple
    Tuples of the form (mar_node0_dic_i,mar_node0_dic_i) where each element in the tuple is a dictionary containing the marginal changes for node0/1 
    for every node0 and node1 involved in a decision (channel open/closures) in the block.
"""


def graph_compare(input_tuple):
    
    block=input_tuple[0]
    key=input_tuple[1]
    block_dec=input_tuple[2]
    block_base=input_tuple[3]
    measurement=input_tuple[4]
    type_dec=input_tuple[5]
    block_res=input_tuple[6]
   
    mar_node0_dic_i={} # dictionary to story function output
    mar_node1_dic_i={} 
    
    
    # Load data
    session = boto3.session.Session()
    s3 = session.resource('s3')
    response = s3.Object(bucket_name=bucket, key=key).get()
    G=pickle.loads(response['Body'].read())
    
    ###########################---------------------
    ##if decisiono 
    
    
    
    
    
    
    
    # For each decision calculate marginal change in measure for node0 and node1
    for edge in block_dec:
        
        # Extract info about channel
        
        node0=edge[0]
        node1=edge[1]
        channel_id=edge[2]['channel_id']
        capacity=edge[2]['capacity']

        
        # Copy original graph
        g_mar=G.copy()   
        old_nodes=False
        
        
        
        # Retrieve base measurement before channel if nodes existed, else define base measure as 0
        if (g_mar.has_node(node0)):
            node0_base=block_base[node0]
            old_nodes=True
        else:
            node0_base=0
            
        if (g_mar.has_node(node1)):
            node1_base=block_base[node1]
            old_nodes=True
        else:
            node1_base=0
        
            
        if old_nodes: # If at least one node is old (part of the connected graph)
            
            if type_dec=='mar_opens': # marginal calculation for opens
                
                
                # Define and add edges and calculate betweeness if at least one of the nodes is in graph 
                edge_list=[edge]
                
                
                # If channel exists increase capacity
                
                if g_mar.has_edge(node0,node1):
                   
                    g_mar.edges[node0,node1]['capacity']+=capacity
                    g_mar.edges[node0,node1]['no_channels']+=1

                else:
                    g_mar.add_edges_from(edge_list)
                
                
                g_mar_mes=node_measurement(g_mar,measurement,node0,node1)
                
                # Update measurement values after marginal change
                node0_new_mes=g_mar_mes[0]
                node1_new_mes=g_mar_mes[1]
            
            elif type_dec=='mar_closures': # marginal calculation for closes
                
                # Define and remove edges, define new connected graph and calculate betweeness 
                edge_list=[(node0,node1)]
                
                
                # If channel exists decrease capacity
                if g_mar.edges[node0,node1]['no_channels']>1:
                    g_mar.edges[node0,node1]['capacity']-=capacity
                    g_mar.edges[node0,node1]['no_channels']-=1
                
                else: 
                    g_mar.remove_edges_from(edge_list) 
                    connected_components=[c for c in nx.algorithms.components.connected_components(g_mar)]
                    g_mar=g_mar.subgraph(connected_components[0]).copy()
                    
                g_mar_mes=node_measurment(g_mar,measurement,node0,node1)
                node0_new_mes=g_mar_mes[0]
                node1_new_mes=g_mar_mes[1]
                
            elif type_dec=='actual': # actual calculation for both opens and closures
                
                # Check individualy if in the graph for the resulting block the node is present (in the connected component, 
                # if not assign measurment to 0. 
                
                try:
                    node0_new_mes=block_res[node0]

                except KeyError:
                    node0_new_mes=0

                try:
                    node1_new_mes=block_res[node1]

                except KeyError:
                    node1_new_mes=0

                
                   
            node0_mar=(node0_new_mes-node0_base)
            node1_mar=(node1_new_mes-node1_base) 
        
        
        else: # If both nodes are new (outside of connected graph) their marginal decision outcome is 0
            node0_mar=0
            node1_mar=0

        
        # Update dictionary - new betweenness
        mar_node0_dic_i[channel_id]=node0_mar
        mar_node1_dic_i[channel_id]=node1_mar
        
    
    return (mar_node0_dic_i,mar_node1_dic_i)

### MARGINAL - CURRENT BETWEENESS

**SCRIPT: Marginal current betweeness for opens**

In [ ]:
# Compute marginal betweeness for channel openings

futures_bet_maropen=collection_compare(blocks,channel_opens,graph_snapshots,snapshot_bet,measurement='current_betweeness',type_dec='mar_opens')
start=time.time()
bet_maropen_diclist = dask.compute(*futures_bet_maropen)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_maropen_diclist)))



In [ ]:
# Write to marginal current betweeness for opens into decisions Dataframe

bet_maropen_channels=add_columns(bet_maropen_diclist,decisions_df,'bet_maropen_node0','bet_maropen_node1')

# Updated DataFrame

print('Rows edited with marginal current betweeness for opens: {}'.format(len(bet_maropen_channels)))
decisions_df[decisions_df['short_channel_id'].isin(bet_maropen_channels)].head(2)


**SCRIPT: Marginal current betweeness for closures**

In [ ]:
# Compute marginal betweeness for channel closures

futures_bet_marclose=collection_compare(blocks,channel_closures,graph_snapshots,snapshot_bet,measurement='current_betweeness',type_dec='mar_closures')
start=time.time()
bet_marclose_diclist = dask.compute(*futures_bet_marclose)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_marclose_diclist)))


In [ ]:
# Write to marginal current betweeness for closures into decisions Dataframe

bet_marclose_channels=add_columns(bet_marclose_diclist,decisions_df,'bet_marclose_node0','bet_marclose_node1')

# Updated DataFrame

print('Rows edited with marginal current betweeness for closures: {}'.format(len(bet_marclose_channels)))
decisions_df[decisions_df['short_channel_id'].isin(bet_marclose_channels)].head(2)

### MARGINAL - CURRENT CLOSENESS

**SCRIPT: Marginal current closeness for opens**

In [ ]:
# Compute marginal betweeness for channel openings

futures_clo_maropen=collection_compare(blocks,channel_opens,graph_snapshots,snapshot_clo,measurement='current_closeness',type_dec='mar_opens')
start=time.time()
clo_maropen_diclist = dask.compute(*futures_clo_maropen)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(clo_maropen_diclist)))


In [ ]:
# Write to marginal current betweeness for opens into decisions Dataframe

clo_maropen_channels=add_columns(clo_maropen_diclist,decisions_df,'clo_maropen_node0','clo_maropen_node1')

# Updated DataFrame

print('Rows edited with marginal current closeness for opens: {}'.format(len(clo_maropen_channels)))
decisions_df[decisions_df['short_channel_id'].isin(clo_maropen_channels)].head(2)


**SCRIPT: Marginal current closeness for closures**

In [ ]:
# Compute marginal current closeness for channel closures

futures_clo_marclose=collection_compare(blocks,channel_closures,graph_snapshots,snapshot_clo,measurement='current_closeness',type_dec='mar_closures')
start=time.time()
clo_marclose_diclist = dask.compute(*futures_clo_marclose)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(clo_marclose_diclist)))

# Write to marginal current betweeness for opens into decisions Dataframe

clo_marclose_channels=add_columns(clo_marclose_diclist,decisions_df,'clo_marclose_node0','clo_marclose_node1')

# Updated DataFrame

print('Rows edited with marginal current closeness for closures: {}'.format(len(clo_marclose_channels)))
decisions_df[decisions_df['short_channel_id'].isin(clo_marclose_channels)].head()

### ACTUAL - CURRENT BETWEENESS

**SCRIPT: Actual current betweeness for opens**

In [ ]:
# Compute actual current betweeness for channel opens

futures_bet_actopen=collection_compare(blocks,channel_opens,graph_snapshots,snapshot_bet,measurement='current_betweeness',type_dec='actual')
start=time.time()
bet_actopen_diclist = dask.compute(*futures_bet_actopen)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_actopen_diclist)))

In [ ]:
# Write to marginal current betweeness for opens into decisions Dataframe

bet_actopen_channels=add_columns(bet_actopen_diclist,decisions_df,'bet_actopen_node0','bet_actopen_node1')

# Updated DataFrame

print('Rows edited with actual current betweeness for opens: {}'.format(len(bet_actopen_channels)))
decisions_df[decisions_df['short_channel_id'].isin(bet_actopen_channels)].head(2)

**SCRIPT: Actual current betweeness for closures**

In [ ]:
# Compute actual current betweeness for channel closures

futures_bet_actclose=collection_compare(blocks,channel_closures,graph_snapshots,snapshot_bet,measurement='current_betweeness',type_dec='actual')
start=time.time()
bet_actclose_diclist = dask.compute(*futures_bet_actclose)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_actclose_diclist)))

In [ ]:
# Write to marginal current betweeness for opens into decisions Dataframe

bet_actclose_channels=add_columns(bet_actclose_diclist,decisions_df,'bet_actclose_node0','bet_actclose_node1')

# Updated DataFrame

print('Rows edited with actual current betweeness for closures: {}'.format(len(bet_actclose_channels)))
decisions_df[decisions_df['short_channel_id'].isin(bet_actclose_channels)].head(2)

### ACTUAL - CURRENT CLOSENESS

**SCRIPT: Actual current closeness for opens**

In [ ]:
# Compute actual current closeness for channel opens

futures_clo_actopen=collection_compare(blocks,channel_opens,graph_snapshots,snapshot_bet,measurement='current_closeness',type_dec='actual')
start=time.time()
clo_actopen_diclist = dask.compute(*futures_clo_actopen)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(bet_actopen_diclist)))

In [ ]:
# Write to marginal current closeness for opens into decisions Dataframe

clo_actopen_channels=add_columns(clo_actopen_diclist,decisions_df,'clo_actopen_node0','clo_actopen_node1')

# Updated DataFrame

print('Rows edited with actual current closeness for opens: {}'.format(len(clo_actopen_channels)))
decisions_df[decisions_df['short_channel_id'].isin(clo_actopen_channels)].head(2)

**SCRIPT: Actual current closeness for closures**

In [ ]:
# Compute actual current closeness for channel closures

futures_clo_actclose=collection_compare(blocks,channel_closures,graph_snapshots,snapshot_bet,measurement='current_closeness',type_dec='actual')
start=time.time()
clo_actclose_diclist = dask.compute(*futures_clo_actclose)
end=time.time()
print('Compute in seconds: {}'.format(end-start))
print('Size in memory: {}'.format(sys.getsizeof(clo_actclose_diclist)))

In [ ]:
# Write to marginal current closeness for opens into decisions Dataframe

clo_actclose_channels=add_columns(clo_actclose_diclist,decisions_df,'clo_actclose_node0','clo_actclose_node1')

# Updated DataFrame

print('Rows edited with actual current closeness for closures: {}'.format(len(clo_actclose_channels)))
decisions_df[decisions_df['short_channel_id'].isin(clo_actclose_channels)].head(2)

# -------------------------------

### Pairwise stability 

- **Marginal betweenness (bet_mar_nodei)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting the decission (adding or removing a channel). Weighted current betweenness centrality is used for this measure.

> **Marginal betweenness for opens** 

**TODO**: Why is length of Dataframe longer than the number of snapshots extracted? Could it be that some channels appear more than once in dataframe?

> **Marginal betweenness for closures** 

- **Actual change in betweenness (bet_act_nodei)**: The % change between the betweenness centrality, for the node under analysis, given the graph from the previous block and the betweenness centrality of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted current betweenness centrality is used for this measure.

- **Marginal betweeness pairwise stability (bet_mar_pairst/open/close)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a betweenness perspective.

In [ ]:
# MARGINAL - Add column with check for pairwise stability compatability using marginal outcomes

# OPEN - Channel is opened if both nodes gain

# Define function 
def bet_pairst_maropen(row):
    if not math.isnan(row['bet_mar_node0']):
        pairst=(row['bet_maropen_node0']>=0 and row['bet_maropen_node1']>=0)
    else:
        pairst=row['bet_maropen_node0']
    return pairst

# Apply function
decisions_df['bet_mar_pairstopen']=decisions_df.apply(bet_pairst_maropen,axis=1)

# CLOSE - Channel is closed if at least one node is better off

# Define function 
def bet_pairst_marclose(row):
    if not math.isnan(row['bet_marclose_node0']):
        pairst=(row['bet_marclose_node0']>0 or row['bet_marclose_node1']>0)
    else:
        pairst=row['bet_marclose_node0']
    return pairst

# Apply function
decisions_df['bet_mar_pairstclose']=decisions_df.apply(bet_pairst_marclose,axis=1)

In [ ]:
# Test MARGINAL OPEN
decisions_df[decisions_df['bet_mar_node0'].notnull()][['bet_mar_node0','bet_mar_node1','bet_mar_pairstopen']].head()

In [ ]:
# Test MARGINAL CLOSE
decisions_df[decisions_df['bet_marclose_node0'].notnull()][['bet_marclose_node0','bet_marclose_node1','bet_mar_pairstclose']].head()

- **Actual betweeness pairwise stability (bet_act_pairstopen/close)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a betweenness perspective. 

In [ ]:
# ACTUAL - Add column with check for pairwise stability compatability using marginal outcomes

# OPEN - Channel is opened if both nodes gain

# Define function 
def bet_pairst_actopen(row):
    if not math.isnan(row['bet_actopen_node0']):
        pairst=(row['bet_actopen_node0']>=0 and row['bet_actopen_node1']>=0)
    else:
        pairst=row['bet_actopen_node0']
    return pairst

# Apply function
decisions_df['bet_act_pairstopen']=decisions_df.apply(bet_pairst_actopen,axis=1)

# CLOSE - Channel is closed if at least one node is better off

# Define function 
def bet_pairst_actclose(row):
    if not math.isnan(row['bet_actclose_node0']):
        pairst=(row['bet_actclose_node0']>0 or row['bet_actclose_node1']>0)
    else:
        pairst=row['bet_actclose_node0']
    return pairst

# Apply function
decisions_df['bet_act_pairstclose']=decisions_df.apply(bet_pairst_actclose,axis=1)

In [ ]:
# Test ACTUAL OPEN
decisions_df[decisions_df['bet_actopen_node0'].notnull()][['bet_actopen_node0','bet_actopen_node1','bet_act_pairstopen']].head()

In [ ]:
# Test ACTUAL CLOSE
decisions_df[decisions_df['bet_actclose_node0'].notnull()][['bet_actclose_node0','bet_actclose_node1','bet_act_pairstclose']].head()

In [ ]:
# Save Updated DataFrame to S3

# Create S3 resource and define values
session = boto3.session.Session()
s3 = session.resource('s3')
csv_buffer = io.StringIO()

# File path and name ([extraction_id]snapshot_bet-[no_blocks]-[start_block]-[end_block])
key_decisions_df='graph_snapshots/'+str(extraction_id)+'_connected/.data_transformations/'+str(extraction_id)+'decisions_df_bet-'+str(no_blocks)+'-'+str(start_block)+'-'+str(end_block)+'.csv'


# Safe DataFrame
decisions_df.to_csv(csv_buffer)
s3.Object(bucket, key_decisions_df).put(Body=csv_buffer.getvalue())


In [ ]:
# Test Save
decisions_df_test_load = s3.Object(bucket_name=bucket, key=key_decisions_df).get()
decisions_df_test=pd.read_csv(io.BytesIO(decisions_df_test_load['Body'].read()),index_col=0)
decisions_df_test==decisions_df

In [ ]:
decisions_df_test.head()

In [ ]:
decisions_df.head()

### Efficiency
- **Average betweeness per block (bet_effic)**: Average betweenness centrality for all the nodes. 

### Nash stability 

- **% Change with respect to not making decision (bet_binstat_deltai)**: The % change in betwewnness centrality, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (bet_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher betweenness centrality. This tells me if my strategy helped me be better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (bet_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher betwneenness centrality. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (bet_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower betwneenness centrality. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).






## Connectivity

### Pairwise stability 

- **Marginal % change in connectivity (con_mar_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting the decission (adding or removing a channel). Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Actual % change in connectivity (con_act_deltai)**: The % change between the shortest path average, for the node under analysis, given the graph from the previous block and the shortest path average of the resulting graph after enacting **all** the decissions (adding or removing a channels) in the current block. Weighted shortest path (_single_source_dijkstra_path_) is used for this measure.

- **Marginal connectivity pairwise stability (con_mar_pairstab)**: Evaluates if given the marginal graph that results from just enacting this decission is consistent with pairwise stability, from a connectivity perspective.

- **Actual connectivity pairwise stability (con_act_pairstab)**: Evaluates if given the marginal graph that results from all the decisions in the block is consitend with pairwise stability, from a connectivity perspective.  



### Nash stability 

- **% Change with respect to not making decision (con_binstat_deltai)**: The % change in shortest path average, for the node under analysis, given the resulting graph after all of the decissions have been executed. 
- **Nash compatible - binary strategy (con_binstat_nash)**: Returns true if given the other decissions enacted in the block not making decision would have NOT have resulted in higher shortest path average. NOTE: This indicates if the strategy selected made the node better off (took into account what others were doing)

(Optional approaches - Check for tracktability)
- **Nash compatible - close only strategy (con_closestat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels would NOT have not resulted in higher shortest path average. (NOTE: Check if there are combinatorial considerations, if so just look at closings up to x) 
- **Nash compatible - close/open (con_allstat_nash)**: Returns true if given the other decissions enacted in the block, closing any other channels (with any node) or opening a channel with one of the round participants would NOT have not resulted in lower shortest path average. (NOTE: To make it reasonable and constraint the strategy space only consider 'similar nodes' or with relationships in the past?).



### Efficiency
- **Average betweeness per block (bet_effic)**: Average shortest path average for all the nodes. 


## Utility Functions

In [ ]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))